# Training of a Random Forest baseline model

### Init config

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

from src.config import (
    DATA_SPLIT_DIR,
    TRAIN_RAW_FILENAME,
    VALIDATION_RAW_FILENAME,
    MODELS_DIR,
)

In [ ]:
ordinal_orders = {
    "GenHlth": ["excellent", "very good", "good", "fair", "poor"],
    "Age": [
        "18-24",
        "25-29",
        "30-34",
        "35-39",
        "40-44",
        "45-49",
        "50-54",
        "55-59",
        "60-64",
        "65-69",
        "70-74",
        "75-79",
        "80+",
    ],
    "Education": [
        "no school",
        "elementary",
        "some high school",
        "high school graduate",
        "college",
        "college graduate",
    ],
    "Income": ["<$10k", "<$15k", "<$20k", "<$25k", "<$35k", "<$50k", "<$75k", ">$75k"],
}

## Pipeline

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

nominal_cols = [
    "HighBP",
    "HighChol",
    "CholCheck",
    "Smoker",
    "Stroke",
    "HeartDiseaseorAttack",
    "PhysActivity",
    "Fruits",
    "Veggies",
    "HvyAlcoholConsump",
    "AnyHealthcare",
    "NoDocbcCost",
    "DiffWalk",
    "Sex",
]
nominal_pipe = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("ohe", OneHotEncoder(handle_unknown="ignore", drop="first")),
    ]
)

ordinal_cols = ["GenHlth", "Age", "Education", "Income"]

ordinal_categories = [ordinal_orders[col] for col in ordinal_cols]

ordinal_pipe = Pipeline(
    [
        ("impute", SimpleImputer(strategy="most_frequent")),
        ("encode", OrdinalEncoder(categories=ordinal_categories)),
    ]
)


numeric_cols = ["BMI", "MentHlth", "PhysHlth"]
num_pipe = Pipeline(
    [("impute", SimpleImputer(strategy="median")), ("scale", StandardScaler())]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipe, numeric_cols),
        ("ord", ordinal_pipe, ordinal_cols),
        ("nom", nominal_pipe, nominal_cols),
    ],
    remainder="drop",
)

classifier = RandomForestClassifier(class_weight="balanced", random_state=0)

pipeline = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("classifier", classifier),
    ]
)


## Load and transform train and validation data

In [ ]:
from sklearn.preprocessing import LabelEncoder


df_train_raw = pd.read_csv(os.path.join(DATA_SPLIT_DIR, TRAIN_RAW_FILENAME))
features_train_raw = df_train_raw.drop("Diabetes_012", axis=1)
target_train_raw = df_train_raw["Diabetes_012"]


df_val_raw = pd.read_csv(os.path.join(DATA_SPLIT_DIR, VALIDATION_RAW_FILENAME))
features_val_raw = df_val_raw.drop("Diabetes_012", axis=1)
target_val_raw = df_val_raw["Diabetes_012"]


labelencoder = LabelEncoder()

target_train_enc = labelencoder.fit_transform(target_train_raw)

target_val_enc = labelencoder.transform(target_val_raw)

## Training

In [ ]:
%%time
from datetime import datetime

pipeline.fit(features_train_raw, target_train_enc)


timestamp = datetime.now()

## Prediction

In [ ]:
target_val_pred = pipeline.predict(features_val_raw)

## Evaluation

### Metrics

In [ ]:
model_purpose = "baseline"
special_features = ""

In [ ]:
from src.model_evaluation import (
    evaluate_classifier,
    get_classification_report_from_results_as_df,
    get_confusion_matrix_from_results_as_df,
)

target_val_pred_proba = None

if hasattr(classifier, "predict_proba"):
    target_val_pred_proba = pipeline.predict_proba(features_val_raw)

    if target_train_raw.nunique() <= 2:
        target_val_pred_proba = target_val_pred_proba[:, 1]

results = evaluate_classifier(
    classifier=classifier,
    labels=list(labelencoder.classes_),
    target_truth=target_val_raw,
    target_pred=labelencoder.inverse_transform(target_val_pred),
    target_pred_proba=target_val_pred_proba,
    timestamp=timestamp,
    model_purpose=model_purpose,
    special_features=special_features,
)


print("precision:", results["precision"])
print("F1:", results["f1"])
print("bal acc:", results["bal_accuracy"])
print("roc auc:", results["roc_auc_score"])

print()
print("confusion_matrix")
display(get_confusion_matrix_from_results_as_df(results))

print()
print("classification_report")
display(get_classification_report_from_results_as_df(results))


labels = results["predicts"]
model_name = results["model_name"]

### Save the model and results

In [ ]:
import pickle
import json
import os

folder = os.path.join(MODELS_DIR, model_name)
filename = os.path.join(folder, model_name)
os.makedirs(folder, exist_ok=True)

with open(f"{filename}.model.pkl", "wb") as f:
    pickle.dump(classifier, f)

with open(f"{filename}.pipeline.pkl", "wb") as f:
    pickle.dump(preprocessor, f)

with open(f"{filename}.label_encoder.pkl", "wb") as f:
    pickle.dump(labelencoder, f)

with open(f"{filename}.model.txt", "w") as file:
    file.write(str(classifier))

with open(f"{filename}.results.json", "w") as f:
    json.dump(results, f, indent=2)

with open(f"{filename}.pipeline_params.txt", "w") as f:
    f.write(preprocessor.get_params().__str__())

with open(f"{filename}.model_params.json", "w") as f:
    json.dump(classifier.get_params(), f, indent=2)


### Learning Curve

In [ ]:
%%time

scoring_mode = "f1_macro"
train_sizes, train_scores, val_scores = learning_curve(
    estimator=classifier,
    X=preprocessor.transform(features_train_raw),
    y=target_train_enc,
    cv=5,
    scoring=scoring_mode,
    n_jobs=-1,
    train_sizes=np.linspace(0.1, 1.0, 10),
)

train_mean = train_scores.mean(axis=1)
val_mean = val_scores.mean(axis=1)

df_lc = pd.DataFrame({"train_sizes": train_sizes, "train_scores": train_mean, "val_scores": val_mean})
df_lc.to_csv(f"{filename}.learning_curves.csv", index=False)

In [ ]:
from scipy.ndimage import uniform_filter1d

fig, ax = plt.subplots(figsize=(16, 5))

# Hauptlinien
train_line, = ax.plot(train_sizes, train_mean, label="Training score")
val_line, = ax.plot(train_sizes, val_mean, label="Validation score")

# Farben extrahieren
train_color = train_line.get_color()
val_color = val_line.get_color()

# Min-/Max-Werte
y_lines_min = [train_mean.min(), val_mean.min()]
y_lines_max = [train_mean.max(), val_mean.max()]

# Hilfslinien
for y in y_lines_min:
    ax.axhline(y, color="red", linestyle="--", linewidth=0.8, alpha=0.2)
for y in y_lines_max:
    ax.axhline(y, color="green", linestyle="--", linewidth=0.8, alpha=0.2)

# Rollender Durchschnitt (Fenstergröße = 3)
train_rolling = uniform_filter1d(train_mean, size=3, mode="nearest")
val_rolling = uniform_filter1d(val_mean, size=3, mode="nearest")

ax.plot(train_sizes, train_rolling, label="Train rolling avg", linestyle=":", linewidth=1.5, color=train_color, alpha=0.5)
ax.plot(train_sizes, val_rolling, label="Val rolling avg", linestyle=":", linewidth=1.5, color=val_color, alpha=0.5)

# y-Ticks
yticks = sorted(set(ax.get_yticks().tolist() + y_lines_min + y_lines_max))
ax.set_yticks(yticks)

ax.set_xlabel("Training set size")
ax.set_ylabel(scoring_mode.replace("_", " ").capitalize())

ax.set_title(f"Learning curve for model '{results["timestamp"]}'")

ax.legend()
fig.tight_layout()
fig.savefig(f"{filename}.learning_curve_img.png", format="png")
plt.show()


### Feature Importances

In [ ]:
import seaborn as sns
from sklearn.metrics import f1_score

preprocessed_features_train = preprocessor.transform(features_train_raw)
tgt_train_pred = classifier.predict(preprocessed_features_train)

f1_base = f1_score(target_train_enc, tgt_train_pred, average='weighted')

importances_of_permutations = []
for feat in features_train_raw.columns:
    features_permutated = features_train_raw.copy()

    permutation_run_scores = []
    for i in range(5):
        series_perm = features_permutated[feat].sample(frac=1, replace=False, random_state=0)
        series_perm.reset_index(drop=True, inplace=True)
        features_permutated[feat] = series_perm
        
        preprocessed_features_permutated = preprocessor.transform(features_permutated)
        f1_permutated = f1_score(target_train_enc, classifier.predict(preprocessed_features_permutated), average='weighted')
        permutation_run_scores.append(f1_base - f1_permutated)

    importances_of_permutations.append(np.mean(permutation_run_scores))

feature_importances = pd.Series(data=importances_of_permutations, index=features_train_raw.columns)
feature_importances.sort_values(ascending=False, inplace=True)

with open(f"{filename}.feature_importances.json", "w") as f:
    json.dump(feature_importances.to_dict(), f, indent=2)

In [ ]:
fig, ax = plt.subplots(figsize=(16, 0.3*len(features_val_raw.columns)))
sns.barplot(x=feature_importances.values, y=feature_importances.index,ax=ax)
ax.set_title(f"feature importances for model '{model_name}'")
for i, (val, label) in enumerate(zip(feature_importances.values, feature_importances.index)):
    ax.text(val, i, f" {val:.4f}", va="center", ha="left", fontsize=8)
    
fig.tight_layout()
fig.savefig(f"{filename}.feature_importances.png", format="png")
plt.show()

## Observations
* On The largest class (_no dia_) the model reaches a high precision and high recall. 
* Class _dia_ is predicted correctly only once.
* Class _dia_ has a very low recall.
* Class _dia_ is almost always classified as class _no dia_.
* Most prediction fall in class _no dia_.
* Balanced Accuracy is below 0.38.
* With increassing set size precision learning curves for training and validation decrease.
* Most important features are `BMI` and `GenHlth`.
* Absolute feature importances generally are not very high ~1.5%.

## Conclusion
* Model is worse than trivial guessing.
* The confusion matrix suggests the model
    * basically predicts the majority class.
    * is not usable for minority class detection.
* The learning curves and generally low feature importances suggest the model underfits the data.

## Next Steps
* Feature Engineering is needed for stronger separation.
* Rebalancing of class sizes. (SMOTE)
* Use other models.